In [1]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime

In [2]:
#import csv
df=pd.read_csv("cab_rides.csv")
weather_df=pd.read_csv("weather.csv")

In [3]:
#first 5 rows in the csv
print("---This is df")
print(df.head())

print("---This is weather_df")
print(weather_df.head())

---This is df
   distance cab_type     time_stamp    destination            source  price  \
0      0.44     Lyft  1544952607890  North Station  Haymarket Square    5.0   
1      0.44     Lyft  1543284023677  North Station  Haymarket Square   11.0   
2      0.44     Lyft  1543366822198  North Station  Haymarket Square    7.0   
3      0.44     Lyft  1543553582749  North Station  Haymarket Square   26.0   
4      0.44     Lyft  1543463360223  North Station  Haymarket Square    9.0   

   surge_multiplier                                    id    product_id  \
0               1.0  424553bb-7174-41ea-aeb4-fe06d4f4b9d7     lyft_line   
1               1.0  4bd23055-6827-41c6-b23b-3c491f24e74d  lyft_premier   
2               1.0  981a3613-77af-4620-a42a-0c0866077d1e          lyft   
3               1.0  c2d88af2-d278-4bfd-a8d0-29ca77cc5512   lyft_luxsuv   
4               1.0  e0126e1f-8ca9-4f2e-82b3-50505a09db9a     lyft_plus   

           name  
0        Shared  
1           Lux  
2     

In [4]:
#add a "date" column for converting time_stamp into a readable Date format in df
df['date'] = pd.to_datetime(df['time_stamp'], unit='ms').dt.date
print(df['date'])

#add a "date" column for converting time_stamp into a readable Date format in weather_df
weather_df['date'] = pd.to_datetime(weather_df['time_stamp'], unit='s').dt.date
print(weather_df['date'])

0         2018-12-16
1         2018-11-27
2         2018-11-28
3         2018-11-30
4         2018-11-29
             ...    
693066    2018-12-01
693067    2018-12-01
693068    2018-12-01
693069    2018-12-01
693070    2018-12-01
Name: date, Length: 693071, dtype: object
0       2018-12-16
1       2018-12-16
2       2018-12-16
3       2018-12-16
4       2018-12-16
           ...    
6271    2018-12-03
6272    2018-12-03
6273    2018-12-03
6274    2018-12-03
6275    2018-12-03
Name: date, Length: 6276, dtype: object


In [5]:
#first 5 rows in the csv
print("---This is df")
print(df.head())

print("---This is weather_df")
print(weather_df.head())

---This is df
   distance cab_type     time_stamp    destination            source  price  \
0      0.44     Lyft  1544952607890  North Station  Haymarket Square    5.0   
1      0.44     Lyft  1543284023677  North Station  Haymarket Square   11.0   
2      0.44     Lyft  1543366822198  North Station  Haymarket Square    7.0   
3      0.44     Lyft  1543553582749  North Station  Haymarket Square   26.0   
4      0.44     Lyft  1543463360223  North Station  Haymarket Square    9.0   

   surge_multiplier                                    id    product_id  \
0               1.0  424553bb-7174-41ea-aeb4-fe06d4f4b9d7     lyft_line   
1               1.0  4bd23055-6827-41c6-b23b-3c491f24e74d  lyft_premier   
2               1.0  981a3613-77af-4620-a42a-0c0866077d1e          lyft   
3               1.0  c2d88af2-d278-4bfd-a8d0-29ca77cc5512   lyft_luxsuv   
4               1.0  e0126e1f-8ca9-4f2e-82b3-50505a09db9a     lyft_plus   

           name        date  
0        Shared  2018-12-16  


In [6]:
#check missing data
print(df.isna().sum())
print(weather_df.isna().sum())

distance                0
cab_type                0
time_stamp              0
destination             0
source                  0
price               55095
surge_multiplier        0
id                      0
product_id              0
name                    0
date                    0
dtype: int64
temp             0
location         0
clouds           0
pressure         0
rain          5382
time_stamp       0
humidity         0
wind             0
date             0
dtype: int64


In [7]:
#check which 'name' got the missing price, it shows all come from "Uber Taxi"
df.groupby(['cab_type', 'name']).count()

distance  time_stamp  destination  source  price  \
cab_type name                                                             
Lyft     Lux              51235       51235        51235   51235  51235   
         Lux Black        51235       51235        51235   51235  51235   
         Lux Black XL     51235       51235        51235   51235  51235   
         Lyft             51235       51235        51235   51235  51235   
         Lyft XL          51235       51235        51235   51235  51235   
         Shared           51233       51233        51233   51233  51233   
Uber     Black            55095       55095        55095   55095  55095   
         Black SUV        55096       55096        55096   55096  55096   
         Taxi             55095       55095        55095   55095      0   
         UberPool         55091       55091        55091   55091  55091   
         UberX            55094       55094        55094   55094  55094   
         UberXL           55096       55096        55096   55096  55096   
         WAV              55096       55096        55096   55096  55096   

                       surge_multiplier     id  product_id   date  
cab_type name                                                      
Lyft     Lux                      51235  51235       51235  51235  
         Lux Black                51235  51235       51235  51235  
         Lux Black XL             51235  51235       51235  51235  
         Lyft                     51235  51235       51235  51235  
         Lyft XL                  51235  51235       51235  51235  
         Shared                   51233  51233       51233  51233  
Uber     Black                    55095  55095       55095  55095  
         Black SUV                55096  55096       55096  55096  
         Taxi                     55095  55095       55095  55095  
         UberPool                 55091  55091       55091  55091  
         UberX                    55094  55094       55094  55094  
         UberXL                   55096  55096       55096  55096  
         WAV                      55096  55096       55096  55096

In [8]:
#create a new dataframe in df to drop the Uber Taxi product, so the dataframe only contains private cars (non-taxi) trips
new_df = df.dropna()
new_df.groupby(['cab_type', 'name']).count()

distance  time_stamp  destination  source  price  \
cab_type name                                                             
Lyft     Lux              51235       51235        51235   51235  51235   
         Lux Black        51235       51235        51235   51235  51235   
         Lux Black XL     51235       51235        51235   51235  51235   
         Lyft             51235       51235        51235   51235  51235   
         Lyft XL          51235       51235        51235   51235  51235   
         Shared           51233       51233        51233   51233  51233   
Uber     Black            55095       55095        55095   55095  55095   
         Black SUV        55096       55096        55096   55096  55096   
         UberPool         55091       55091        55091   55091  55091   
         UberX            55094       55094        55094   55094  55094   
         UberXL           55096       55096        55096   55096  55096   
         WAV              55096       55096        55096   55096  55096   

                       surge_multiplier     id  product_id   date  
cab_type name                                                      
Lyft     Lux                      51235  51235       51235  51235  
         Lux Black                51235  51235       51235  51235  
         Lux Black XL             51235  51235       51235  51235  
         Lyft                     51235  51235       51235  51235  
         Lyft XL                  51235  51235       51235  51235  
         Shared                   51233  51233       51233  51233  
Uber     Black                    55095  55095       55095  55095  
         Black SUV                55096  55096       55096  55096  
         UberPool                 55091  55091       55091  55091  
         UberX                    55094  55094       55094  55094  
         UberXL                   55096  55096       55096  55096  
         WAV                      55096  55096       55096  55096

In [9]:
#create a new dataframe in weather_df to fill the missing rain value as 0
new_weather_df = weather_df.fillna(0)
print(new_weather_df.isna().sum())
print(new_weather_df.head())

temp          0
location      0
clouds        0
pressure      0
rain          0
time_stamp    0
humidity      0
wind          0
date          0
dtype: int64
    temp            location  clouds  pressure    rain  time_stamp  humidity  \
0  42.42            Back Bay     1.0   1012.14  0.1228  1545003901      0.77   
1  42.43         Beacon Hill     1.0   1012.15  0.1846  1545003901      0.76   
2  42.50   Boston University     1.0   1012.15  0.1089  1545003901      0.76   
3  42.11              Fenway     1.0   1012.13  0.0969  1545003901      0.77   
4  43.13  Financial District     1.0   1012.14  0.1786  1545003901      0.75   

    wind        date  
0  11.25  2018-12-16  
1  11.32  2018-12-16  
2  11.07  2018-12-16  
3  11.09  2018-12-16  
4  11.49  2018-12-16  


In [10]:
print(new_df.head())
print(new_df.isna().sum())

   distance cab_type     time_stamp    destination            source  price  \
0      0.44     Lyft  1544952607890  North Station  Haymarket Square    5.0   
1      0.44     Lyft  1543284023677  North Station  Haymarket Square   11.0   
2      0.44     Lyft  1543366822198  North Station  Haymarket Square    7.0   
3      0.44     Lyft  1543553582749  North Station  Haymarket Square   26.0   
4      0.44     Lyft  1543463360223  North Station  Haymarket Square    9.0   

   surge_multiplier                                    id    product_id  \
0               1.0  424553bb-7174-41ea-aeb4-fe06d4f4b9d7     lyft_line   
1               1.0  4bd23055-6827-41c6-b23b-3c491f24e74d  lyft_premier   
2               1.0  981a3613-77af-4620-a42a-0c0866077d1e          lyft   
3               1.0  c2d88af2-d278-4bfd-a8d0-29ca77cc5512   lyft_luxsuv   
4               1.0  e0126e1f-8ca9-4f2e-82b3-50505a09db9a     lyft_plus   

           name        date  
0        Shared  2018-12-16  
1           Lu

In [23]:
#add the date & source location in both new_df & weather_df for understanding the order placed location's weather of the day
new_df["source_date"] = new_df.source.astype("str") + " " + new_df.date.astype("str")

/var/folders/6b/yqndd57501bgqdg0b5qjzz_w0000gn/T/ipykernel_27113/3982383052.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["source_date"] = new_df.source.astype("str") + " " + new_df.date.astype("str")


In [12]:
#add the date & source location in both new_df & weather_df for understanding the order placed location's weather of the day
new_weather_df["source_date"] = new_weather_df.location.astype("str") + " " + new_weather_df.date.astype("str")
new_weather_df["source_date"]

0                      Back Bay 2018-12-16
1                   Beacon Hill 2018-12-16
2             Boston University 2018-12-16
3                        Fenway 2018-12-16
4            Financial District 2018-12-16
                       ...                
6271              North Station 2018-12-03
6272    Northeastern University 2018-12-03
6273              South Station 2018-12-03
6274           Theatre District 2018-12-03
6275                   West End 2018-12-03
Name: source_date, Length: 6276, dtype: object

In [20]:
#merge new_df & new_weather_df dataset to have the start locations' weather conditions
new_weather_df.index = new_weather_df['source_date']
merge_df = new_df.join(new_weather_df, on=['source_date'], rsuffix="_right")
merge_df.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,...,temp,location,clouds,pressure,rain,time_stamp_right,humidity,wind,date_right,source_date_right
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,...,42.34,Haymarket Square,1.00,1012.15,0.2068,1545003901,0.77,11.49,2018-12-16,Haymarket Square 2018-12-16
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,...,41.68,Haymarket Square,0.44,1023.68,0.0000,1544924701,0.71,6.28,2018-12-16,Haymarket Square 2018-12-16
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,...,42.75,Haymarket Square,0.17,1023.04,0.0000,1544921101,0.71,6.45,2018-12-16,Haymarket Square 2018-12-16
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,...,40.99,Haymarket Square,0.40,1024.49,0.0000,1544928301,0.71,6.38,2018-12-16,Haymarket Square 2018-12-16
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,...,40.15,Haymarket Square,0.41,1023.50,0.0000,1544939101,0.70,7.82,2018-12-16,Haymarket Square 2018-12-16


In [24]:
def mean(column):
    """Get the mean in a column"""
    mean = sum(column) / len (column)
    return mean

In [29]:
mean(merge_df['temp'][])

39.3139754626732

In [14]:
np_name = np.array(new_df['name'])
np_price = np.array(new_df['price'])
np_cabtype = np.array(new_df['cab_type'])
np_distance = np.array(new_df['distance'])

In [15]:
#Calculate the price per distance of Uber & Lyft (Using Numpy Array)
uber_price = np_price[np_cabtype == 'Uber']
uber_distance = np_distance[np_cabtype == 'Uber']
uber_price_per_distance = uber_price.sum() / uber_distance.sum()
print("Uber Price per Distance = "+ str(uber_price_per_distance))

lyft_price = np_price[np_cabtype == 'Lyft']
lyft_distance = np_distance[np_cabtype == 'Lyft']
lyft_price_per_distance = lyft_price.sum() / lyft_distance.sum()
print("Lyft Price per Distance = "+ str(lyft_price_per_distance))

if uber_price_per_distance > lyft_price_per_distance:
    print("Uber got a higher price per distance than Lyft")
elif uber_price_per_distance == lyft_price_per_distance:
    print("Uber & Lyft got the same price per distance")
else:
    print("Lyft got a higher price per distance than Uber")

Uber Price per Distance = 7.207922117856482
Lyft Price per Distance = 7.933968838236628
Lyft got a higher price per distance than Uber


In [16]:
#create product list of each cab_type
uber_product = ['Black', 'Black SUV', 'UberPool', 'UberX', 'UberXL', 'WAV']
lyft_product = ['Lux', 'Lux Black', 'Lux Black XL', 'Lyft', 'Lyft XL', 'Shared']

In [17]:
sns.scatterplot(x=cab_type, y=price, data=df, hue='uber_product')

NameError: name 'cab_type' is not defined

In [ ]:
#custom function
def add_numbers(a,b):
   sum = a + b
   return sum

num1 = 5
num2 = 6

print("The sum is", add_numbers(num1, num2))

In [ ]:
new_df.describe()

In [ ]:
df.info()

In [ ]:
#check the list of product type
df.product_id.unique()

In [ ]:
sns.catplot(x="cab_type", y=Uber, Lyft, data=df, kind="bar")
plt.show()

In [ ]:
#some of the products don't contains the brand, which may difficult to identify the product belongs to which brand, so add one more column to include the brand name into the product
df["product_name"] = df["cab_type"] + "-" + df["name"]
print(df.product)

In [ ]:
df.product_name.unique()

In [ ]:
uber_regex=r"\w+-\w+-\w+-\w+-\w+"

In [ ]:
def mean(values):
#Write the mean() function
mean = sum(values) / len(values)
return mean

In [ ]:
df_new["price"] = df_new["price"].fillna(method="bfill")

In [ ]:
df_new.isna().sum()

In [ ]:
# Create a dictionary mapping subgroup values to colors
palette_colors = {"Uber": "black", "Lyft": "magenta"}

In [ ]:
sns.countplot(x="cab_type", data=df_new, palette=palette_colors)

In [ ]:
Uber=df_new[df_new['cab_type']=='Uber']

In [ ]:
Lyft=df_new[df_new['cab_type']=='Lyft']

In [ ]:
df_new.info()

In [ ]:
np_price_per_mile = np.array(df_new['price']) / np.array(df_new['distance'])

In [ ]:
print(np_price_per_mile)

In [ ]:
print(df_new['price'] / df_new['distance'])